<a href="https://colab.research.google.com/github/Varchala/Secure_Private_AI/blob/main/Federated_Learning_varchala.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install tensorflow_federated==0.13.1 --quiet

In [ ]:
!pip install -U git+https://github.com/tensorflow/privacy --quiet

In [ ]:
# !pip install --upgrade tensorflow --quiet

In [5]:
# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
# !pip uninstall --yes tensorboard tb-nightly

# !pip install --quiet --upgrade tensorflow-federated-nightly
# !pip install --quiet --upgrade nest-asyncio
# !pip install --quiet --upgrade tb-nightly  # or tensorboard, but not both


In [1]:

import nest_asyncio
nest_asyncio.apply()

In [2]:
%load_ext tensorboard

In [3]:
import collections
import time
import numpy as np
import tensorflow as tf
import tensorflow_federated as tff
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd
import numpy as np
from tensorflow.keras import utils as np_utils
import tensorflow.keras.backend as K
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
# from keras.optimizers import Adam
from tensorflow.keras.layers import InputLayer, Reshape
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import CategoricalCrossentropy
np.random.seed(0)
import os
import random
from tqdm import tqdm
import numpy as np
import torch, torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data.dataset import Dataset   
torch.backends.cudnn.benchmark=True
from torch.utils.data import DataLoader, Dataset
tff.federated_computation(lambda: 'Hello, World!')()

b'Hello, World!'

# Next version

In [4]:
(x_train_100, y_train_100), (x_test_100, y_test_100) = cifar100.load_data()
# y_test = to_categorical(y_test, 100)
# y_train = to_categorical(y_train, 100)
x_train_100 = x_train_100.astype('float32')
x_test_100 = x_test_100.astype('float32')
x_train_100  /= 255
x_test_100 /= 255
# print(y_train.shape)
# assert x_train_100.shape == (50000, 32, 32, 3)
# assert x_test.shape == (10000, 32, 32, 3)
# assert y_train.shape == (50000, 100)
# assert y_test.shape == (10000, 100)

169017344/169001437 [==============================] - 3s 0us/step


In [5]:
x_train_100.shape

(50000, 32, 32, 3)

In [6]:
y_train_100.flatten()

array([19, 29,  0, ...,  3,  7, 73])

In [7]:
def extract_n_classes( data, labels,n, cat_y = True):
    data_cl=[]
    y=[]
    # print(data.shape)
    for i in range(n):
            # print(data[ np.argwhere( labels == i ).reshape(-1) ].shape)
            # print(labels.shape)
            # print(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1)].shape,data.shape)
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))

    # print(np.array(data_cl).shape)
    x = np.vstack( (data_cl) )
    y = np.array(y)
    # print("In extract classes function")
    # print(x.shape,y.shape)
    return x, y

In [8]:
#extract equal ratio of all the classes for train
x_train, y_train = extract_n_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [9]:
#extract equal ratio of all the classes for test
x_test, y_test = extract_n_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [10]:
x_test.shape,y_test.shape

((10000, 32, 32, 3), (10000,))

In [11]:
x_test = x_test.reshape(-1,3072)

In [12]:
x_test.shape

(10000, 3072)

In [13]:
np.unique(y_test, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100,
        100, 100, 100, 100, 100, 100, 1

In [14]:
x_train.shape

(50000, 32, 32, 3)

In [ ]:
# from collections import defaultdict
# dataset={}
# dataset['image'] = data.reshape(-1,3072)
# dataset['label'] = label

In [15]:
x_train = x_train.reshape(-1,3072)

In [16]:
x_train.shape

(50000, 3072)

In [17]:
np.unique(y_train, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
        34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
        51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67,
        68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84,
        85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
 array([500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500, 500,
        500, 500, 500, 500, 500, 500, 5

## 1) Every party only holds instances of one class

### 1) 10 Clients

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
y_train[499]

0

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:

# parameters
NO_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 32
EPOCHS = 20

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.06882999837398529,loss=4.302463531494141,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.12751999497413635,loss=3.902362585067749,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.1748799979686737,loss=3.5745668411254883,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.21015000343322754,loss=3.371450424194336,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.24036000669002533,loss=3.214587450027466,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.2699599862098694,loss=3.0833005905151367,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.2941400110721588,loss=2.9663169384002686,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.3153899908065796,loss=2.861238479614258,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_accuracy=0.33

### Evaluation

In [ ]:
x_test.shape[ 0 ]//100

100

In [ ]:
NO_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.14399999380111694,loss=4.153222560882568,keras_training_time_client_sum_sec=0.0>


### 2) 20 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = extract_n_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = extract_n_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)

In [ ]:
x_train = x_train.reshape(-1,3072)

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
y_train[499]

0

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:

# parameters
NO_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 32
EPOCHS = 20

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.9968000054359436,loss=0.02838919870555401,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9968699812889099,loss=0.021680328994989395,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9969499707221985,loss=0.01897040754556656,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.9969850182533264,loss=0.016871197149157524,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.996999979019165,loss=0.015239870175719261,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.9969900250434875,loss=0.01391354575753212,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.9970600008964539,loss=0.012825400568544865,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.9970999956130981,loss=0.01204193476587534,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_a

### Evaluation

In [ ]:
x_test.shape[ 0 ]//100

100

In [ ]:
NO_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.14149999618530273,loss=2.9097464084625244,keras_training_time_client_sum_sec=0.0>


### 3) 50 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = extract_n_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = extract_n_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)

In [ ]:
x_train = x_train.reshape(-1,3072)

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
y_train[499]

0

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:

# parameters
NO_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 32
EPOCHS = 20

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.9967820048332214,loss=0.029505418613553047,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9968259930610657,loss=0.025112411007285118,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9968720078468323,loss=0.023170992732048035,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.996891975402832,loss=0.021677041426301003,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.9969019889831543,loss=0.02052840031683445,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.9968960285186768,loss=0.01954890787601471,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.9969000220298767,loss=0.01855125091969967,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.9969019889831543,loss=0.017708012834191322,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_

### Evaluation

In [ ]:
x_test.shape[ 0 ]//100

100

In [ ]:
NO_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.04259999841451645,loss=3.9147067070007324,keras_training_time_client_sum_sec=0.0>


### 3) 100 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train, y_train = extract_n_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = extract_n_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_test = x_test.reshape(-1,3072)

In [ ]:
x_train = x_train.reshape(-1,3072)

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
y_train[499]

0

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:

# parameters
NO_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
BATCH_SIZE = 32
EPOCHS = 20

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.9968159794807434,loss=0.031106669455766678,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.9968420267105103,loss=0.02494497410953045,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.9968379735946655,loss=0.023496944457292557,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.9968410134315491,loss=0.02237214520573616,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.996849000453949,loss=0.021402334794402122,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.9968439936637878,loss=0.02064037136733532,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.9968299865722656,loss=0.019981471821665764,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.9968360066413879,loss=0.01941506192088127,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_a

### Evaluation

In [ ]:
x_test.shape[ 0 ]//100

100

In [22]:
y_test[0:100]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
NO_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // 100 # number of samples per client
data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ].reshape(100,1,3072)
  print( x.shape )
  y = y_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.014700000174343586,loss=4.6707353591918945,keras_training_time_client_sum_sec=0.0>


In [ ]:
state, metrics = algorithm.next( state, data )

In [ ]:
# algorithm.

In [ ]:
metrics

AnonymousTuple([('sparse_categorical_accuracy', 0.9904), ('loss', 0.04821914), ('keras_training_time_client_sum_sec', 0.0)])

In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.0142000000923872,loss=4.6595377922058105,keras_training_time_client_sum_sec=0.0>


## 1) Every party has instances of every class

In [18]:
def extract_n_classes( data, labels,n, cat_y = True):
    data_cl=[]
    y=[]
    # print(data.shape)
    for i in range(n):
            # print(data[ np.argwhere( labels == i ).reshape(-1) ].shape)
            # print(labels.shape)
            # print(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1)].shape,data.shape)
            data_cl.append(data[ np.argwhere( labels.reshape(-1) == i ).reshape(-1) ][ : ])
            y.extend(np.full((data_cl[i].shape[0]), i, dtype=int))

    # print(np.array(data_cl).shape)
    x = np.vstack( (data_cl) )
    y = np.array(y)
    # print("In extract classes function")
    # print(x.shape,y.shape)
    return x, y

In [19]:
#extract equal ratio of all the classes for train
x_train, y_train = extract_n_classes( x_train_100, y_train_100 ,n=100,cat_y=False)
session = tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [20]:
x_test = x_test.reshape(-1,3072)
x_train = x_train.reshape(-1,3072)

In [21]:
x_train.shape

(50000, 3072)

### Following function arranges data such that the examples corresponsing to the labels come is 0,1,2,3,4,.....99,0,1,2,3,......99.......99


In [22]:

x_new = []
y_new = []
for i in range(500):
    for j in range(x_train.shape[0]//500):
        x_new.append(x_train[j*500+i])
        y_new.append(y_train[j*500+i])

In [23]:
x_new = np.array(x_new)
x_new.shape

(50000, 3072)

In [24]:
y_new = np.array(y_new)
y_new.shape

(50000,)

In [25]:
print(y_new[0:100])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71
 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95
 96 97 98 99]


### 1) 10 Clients

In [26]:
x_new.shape[ 0 ]

50000

In [27]:
y_new[501]

1

In [28]:
x_new.shape[ 0 ]//100

500

In [29]:
x_train = x_new.copy()
y_train = y_new.copy()

In [30]:
%%time
# parameters
NO_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
BATCH_SIZE = 32
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(5000, 3072)
(5000, 1)
<BatchDataset shapes: ((None, 30

### Evaluation

In [31]:
x_test.shape

(10000, 3072)

In [32]:
x_test.shape[ 0 ]//100

100

In [33]:
NO_CLIENTS = 10 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ].reshape(NO_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(1000, 1, 3072)
(1000, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(

In [34]:
evaluation = tff.learning.build_federated_evaluation(model_function)

In [35]:
test_metrics = evaluation(state.model, data)

In [36]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.20250000059604645,loss=3.4426770210266113,keras_training_time_client_sum_sec=0.0>


### 2) 20 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = extract_n_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
y_train[499]

99

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:

# parameters
NO_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
BATCH_SIZE = 32
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(2500, 3072)
(2500, 1)
<BatchDataset shapes: ((None, 30


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.03213400021195412,loss=4.477772235870361,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.07795400172472,loss=4.09898042678833,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.11436200141906738,loss=3.8583664894104004,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.13838399946689606,loss=3.7215287685394287,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.15478600561618805,loss=3.6306722164154053,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.1678059995174408,loss=3.562695026397705,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.17857399582862854,loss=3.5056324005126953,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.18853400647640228,loss=3.455397605895996,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_accuracy=0.19

### Evaluation

In [ ]:
x_test.shape[ 0 ]//100

100

In [ ]:
NO_CLIENTS = 20 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ].reshape(NO_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(500, 1, 3072)
(500

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.18209999799728394,loss=3.551243305206299,keras_training_time_client_sum_sec=0.0>


### 3) 50 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = extract_n_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
y_train[499]

99

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:

# parameters
NO_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
BATCH_SIZE = 32
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(1000, 3072)
(1000, 1)
<BatchDataset shapes: ((None, 30


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



epoch 0 <sparse_categorical_accuracy=0.04899799823760986,loss=4.396090030670166,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.11541800200939178,loss=3.9218549728393555,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.1563519984483719,loss=3.646138906478882,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.1839890033006668,loss=3.4922215938568115,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.2050500065088272,loss=3.3796703815460205,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.22156600654125214,loss=3.2927358150482178,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.23540900647640228,loss=3.2207276821136475,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.24752900004386902,loss=3.160170316696167,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_accuracy=0

KeyboardInterrupt: ignored

### Evaluation

In [ ]:
x_test.shape[ 0 ]//100

100

In [ ]:
NO_CLIENTS = 50 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ].reshape(NO_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(200, 1, 3072)
(200

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.1793999969959259,loss=3.5994575023651123,keras_training_time_client_sum_sec=0.0>


### 4) 100 Clients

In [ ]:
#extract equal ratio of all the classes for train
x_train = x_new.copy()
y_train = y_new.copy()

In [ ]:
#extract equal ratio of all the classes for test
x_test, y_test = extract_n_classes( x_test_100, y_test_100 ,n=100,cat_y=False)
session =  tf.compat.v1.Session()
tf.compat.v1.keras.backend.set_session( session )

In [ ]:
x_train.shape[ 0 ]

50000

In [ ]:
y_train[499]

99

In [ ]:
x_train.shape[ 0 ]//100

500

In [ ]:

# parameters
NO_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_train.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
BATCH_SIZE = 32
EPOCHS = 10

# shuffle data
# idx = np.arange( TOTAL_SAMPLES )
# np.random.shuffle( idx )
# x_train = x_train[ idx ]
# y_train = y_train[ idx ]

data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ]
  print( x.shape )
  y = y_train[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( [-1,1] ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
  ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )


# define a function that builds our model
def build_model():
  model = tf.keras.models.Sequential()

  model.add( tf.keras.layers.Dense( 64, activation='relu', input_shape=( x_train.shape[ 1: ] ) ) )
  model.add( tf.keras.layers.Dense( 100, activation='softmax' ) )

  return model

def model_function():
  # we need a dummy batch to build the federated model
  # From the docs:
  # A nested structure of values that are convertible to batched tensors
  # with the same shapes and types as expected by forward_pass(). 
  # The values of the tensors are not important and can be filled with any 
  # reasonable input value.
  dummy_batch = collections.OrderedDict( [ 
      ('x', np.ones( ( BATCH_SIZE, x_train.shape[ 1 ] ) ) ),
      ('y', np.ones( ( BATCH_SIZE, 1) ) ) ] )

  # get the compiled keras model
  model = build_model()
  # use tensorflow function to create a federated learning model
  return tff.learning.from_keras_model( model, loss=tf.keras.losses.SparseCategoricalCrossentropy(),  dummy_batch=dummy_batch, metrics=[tf.keras.metrics.SparseCategoricalAccuracy() ] )


# use tensorflow to create the averaging algorithm
algorithm = tff.learning.build_federated_averaging_process( model_function, client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.02 ) ) 

# initialize the learning algorithm and get the initial state
state = algorithm.initialize()

# run the training steps
start = time.time()
for e in range( EPOCHS ):
    state, metrics = algorithm.next( state, data )
    print( 'epoch' , e , metrics )
stop = time.time()
print(f"Training time: {stop - start}s")


(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), types: (tf.float64, tf.float64)>
(500, 3072)
(500, 1)
<BatchDataset shapes: ((None, 3072), (None, 1)), typ


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



ERROR:asyncio:Task was destroyed but it is pending!
task: <Task pending coro=<trace.<locals>.async_trace() running at /usr/local/lib/python3.7/dist-packages/tensorflow_federated/python/common_libs/tracing.py:281> wait_for=<Future pending cb=[_chain_future.<locals>._call_check_cancel() at /usr/lib/python3.7/asyncio/futures.py:351, Task.__wakeup()]> cb=[Task.__wakeup()]>


epoch 0 <sparse_categorical_accuracy=0.0243539996445179,loss=4.5499677658081055,keras_training_time_client_sum_sec=0.0>
epoch 1 <sparse_categorical_accuracy=0.03557400032877922,loss=4.488467216491699,keras_training_time_client_sum_sec=0.0>
epoch 2 <sparse_categorical_accuracy=0.04927999898791313,loss=4.385908603668213,keras_training_time_client_sum_sec=0.0>
epoch 3 <sparse_categorical_accuracy=0.06531199812889099,loss=4.244858264923096,keras_training_time_client_sum_sec=0.0>
epoch 4 <sparse_categorical_accuracy=0.08216000348329544,loss=4.096020221710205,keras_training_time_client_sum_sec=0.0>
epoch 5 <sparse_categorical_accuracy=0.09690000116825104,loss=3.9840710163116455,keras_training_time_client_sum_sec=0.0>
epoch 6 <sparse_categorical_accuracy=0.10847199708223343,loss=3.901385545730591,keras_training_time_client_sum_sec=0.0>
epoch 7 <sparse_categorical_accuracy=0.1179099977016449,loss=3.838557243347168,keras_training_time_client_sum_sec=0.0>
epoch 8 <sparse_categorical_accuracy=0.1

### Evaluation

In [ ]:
x_test.shape[ 0 ]//100

100

In [ ]:
NO_CLIENTS = 100 # number of clients
TOTAL_SAMPLES = x_test.shape[ 0 ]
NO_CLIENT_SAMPLES = TOTAL_SAMPLES // NO_CLIENTS # number of samples per client
data = []
# split into clients
for i in range( NO_CLIENTS ):
  x = x_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES  ].reshape(NO_CLIENT_SAMPLES,1,3072)
  print( x.shape )
  y = y_test[ i * NO_CLIENT_SAMPLES : ( i + 1 ) * NO_CLIENT_SAMPLES ].reshape( -1,1,1 ) 
  print( y.shape )
  ds = tf.data.Dataset.from_tensor_slices( (x.astype( np.float ) , y.astype( np.float ) ) )
#   ds = ds.repeat( EPOCHS ).shuffle( 200 ).batch( BATCH_SIZE )

  print( ds )
  data.append( ds )

(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100, 1, 1)
<TensorSliceDataset shapes: ((1, 3072), (1, 1)), types: (tf.float64, tf.float64)>
(100, 1, 3072)
(100

In [ ]:
evaluation = tff.learning.build_federated_evaluation(model_function)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.




If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [ ]:
test_metrics = evaluation(state.model, data)

In [ ]:
print(str(test_metrics))

<sparse_categorical_accuracy=0.12060000002384186,loss=3.891695261001587,keras_training_time_client_sum_sec=0.0>


# Privacy Attack

## I am using the 10 clients model from every party has instances of every class model.

In [52]:
tff.__version__

'0.13.1'

In [60]:
# state.model.non_trainable.summary()
state.model.trainable[3]

array([-0.23248132, -0.0125431 , -0.14318414,  0.250688  ,  0.38469976,
       -0.23434952, -0.1126335 ,  0.01317185,  0.0620439 , -0.17180759,
       -0.19698215, -0.11864652, -0.01803164,  0.08926512,  0.18667966,
       -0.04394913, -0.19491448, -0.14071274,  0.05890048,  0.15100561,
       -0.4371449 ,  0.4994448 , -0.25189635, -0.17176385, -0.31817666,
       -0.11166035,  0.10329451,  0.26682028, -0.11757993, -0.02721741,
        0.24795066,  0.27294898, -0.03678288,  0.4233782 ,  0.01397543,
       -0.10276289, -0.35437813, -0.05042579,  0.0434991 , -0.08308277,
       -0.20776892, -0.31029722,  0.3212758 ,  0.0909095 ,  0.0762846 ,
        0.17890659,  0.11688788,  0.01461449,  0.13094157, -0.12350072,
        0.00438271,  0.15840597, -0.02025284, -0.36373484, -0.18338042,
        0.26968583,  0.04767029, -0.3036686 ,  0.02351335,  0.05479702,
       -0.27954268, -0.41466045, -0.1750093 ,  0.24549101,  0.2274905 ,
        0.01920389,  0.3251614 , -0.04381545, -0.16401173, -0.20

In [37]:
# tensorflow-privacy 
from tensorflow_privacy.privacy.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackInputData
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import SlicingSpec
from tensorflow_privacy.privacy.membership_inference_attack.data_structures import AttackType

/usr/local/lib/python3.7/dist-packages/tensorflow_privacy/privacy/membership_inference_attack/__init__.py:19: UserWarning: 
Membership inference attack sources were moved. Please replace
import tensorflow_privacy.privacy.membership_inference_attack

with
import tensorflow_privacy.privacy.privacy_tests.membership_inference_attack
  "\nMembership inference attack sources were moved. Please replace"


In [ ]:
# since we have not specified an activation function on the last layer
# calling the predict function returns the logits
print('Predict on train...')
logits_train = model.predict(train_data)
print('Predict on test...')
logits_test = model.predict(test_data)

print('Apply softmax to get probabilities from logits...')
prob_train = special.softmax(logits_train, axis=1)
prob_test = special.softmax(logits_test, axis=1)

print('Compute losses...')
cce = tf.keras.backend.categorical_crossentropy
constant = tf.keras.backend.constant

y_train_onehot = to_categorical(train_labels)
y_test_onehot = to_categorical(test_labels)

loss_train = cce(constant(y_train_onehot), constant(prob_train), from_logits=False).numpy()
loss_test = cce(constant(y_test_onehot), constant(prob_test), from_logits=False).numpy()

In [ ]:
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack import membership_inference_attack as mia
from tensorflow_privacy.privacy.privacy_tests.membership_inference_attack.data_structures import AttackInputData

# Suppose we have the labels as integers starting from 0
# labels_train  shape: (n_train, )
# labels_test  shape: (n_test, )

# Evaluate your model on training and test examples to get
# loss_train  shape: (n_train, )
# loss_test  shape: (n_test, )

# define what variables our attacker should have access to
logits = state.model.trainable[3]
attack_input = AttackInputData(
  labels_train = y_train,
  labels_test = y_test
)
# how should the data be sliced
slicing_spec = SlicingSpec(
    entire_dataset = True,
    by_class = True,
    by_percentiles = False,
    by_classification_correctness = False)
# define the type of attacker model that we want to use
attack_types = [
    AttackType.THRESHOLD_ATTACK
]
# print(attack_types)
attacks_result = mia.run_attacks(attack_input=attack_input,
                                 slicing_spec=slicing_spec,
                                 attack_types=attack_types)

print(attacks_result.summary())


In [6]:

# num = input ("Enter number :") 
# print(num) 
# name1 = input("Enter name : ") 
# print(name1) 

# # Printing type of input value 
# print ("type of number", type(num)) 
# print ("type of name", type(name1)) 